Setup


In [2]:
# === Cell 0: Imports ===
import os, re, glob
from pathlib import Path
import numpy as np
import pandas as pd
import soundfile as sf
import librosa
import matplotlib.pyplot as plt
from numpy.fft import rfft, rfftfreq

print("Ready!")


Ready!


1. Analysis Parameters


In [3]:
# === Cell 1: Tunable parameters ===
SR        = 48000            # analysis sample rate
F0        = 1000.0           # test tone (Hz)
NFFT_MAIN = 65536            # FFT size (tone/silence spectra)
BAND_HZ   = 5.0              # ±Hz around f0/harmonics to integrate power
TRIM_SEC  = 1.0              # cut head+tail (avoid transients)
MAX_H     = 10               # up to 10th harmonic

# Short-time SNR (per-frame)
ST_FRAME  = int(0.050*SR)    # 50 ms
ST_HOP    = int(0.025*SR)    # 25 ms
NFFT_ST   = 16384

DATA_ROOT = Path("data")     # Step-1 output folder
PLOTS_DIR = Path("plots_step2"); PLOTS_DIR.mkdir(parents=True, exist_ok=True)
